In [ ]:
import json

import matplotlib.pyplot as plt

from octopus_dash import data

In [ ]:
with open("../donttrack/config.json") as tf:
    config = json.load(tf)

In [ ]:
d = data\
        .data_getter(**config)\
        .load_electric_data()

In [ ]:
df = d.enrich_data().get_dataframe()

In [ ]:
df

In [ ]:
df.groupby("hour")["consumption"]\
    .mean()\
    .plot(
        kind = "bar", 
        figsize = (10, 6)
)

In [ ]:
d.get_monthly_cost()

In [ ]:
d.tariff.off_peak_rate = 12.00/100
d.tariff.peak_rate = 39.95/100
d.standing_day_charge = 49.11/100

In [ ]:
d.get_monthly_cost()

In [ ]:
import pandas as pd

In [ ]:
df = df.assign(interval_start = lambda x: pd.to_datetime(x.interval_start, utc = True))

In [ ]:
df.set_index("interval_start", inplace=True)

In [ ]:
df.groupby(df.index.month)["off_grid"].mean().plot(kind="bar")

In [ ]:
df.groupby([df.index.month, df.off_peak])["cost"].sum().unstack().loc[3:].plot(kind="area")

In [ ]:
df.groupby(df.index.month)["off_grid"].mean().plot(kind="bar")